In [1]:
# prompt: https://s3.amazonaws.com/google-landmark/train/images_000.tar access the images in this and load thenm into a folder

# Accessing a .tar file from a URL and extracting images into a folder

import requests
import tarfile
import os

# URL of the tar file
url = "https://s3.amazonaws.com/google-landmark/train/images_000.tar"

# Destination folder to extract images
extract_folder = "landmark_images"

# Create the extraction folder if it doesn't exist
if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

# Download the tar file
try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Extract the tar file
    with tarfile.open(fileobj=response.raw, mode="r|*") as tar:
        # Extract all contents into the specified folder
        tar.extractall(path=extract_folder)
    print(f"Successfully downloaded and extracted images to '{extract_folder}'")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")
except tarfile.TarError as e:
    print(f"Error extracting the tar file: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# You can now access the images in the 'landmark_images' folder.
# For example, to list the files in the folder:
# !ls {extract_folder}

Successfully downloaded and extracted images to 'landmark_images'


In [2]:
import os
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, applications

# 1. Define your Base Path (same as before)
base_path = "landmark_images" # Updated base path to the extracted images folder

# Load your training data (same as before)
try:
    df = pd.read_csv("/content/train.csv")
    # Modified to use 'id' column and handle potential errors
    df['image_id_str'] = df['id'].apply(lambda x: "00" + str(x).split(' ')[0] if pd.notna(x) else None)
    df = df.dropna(subset=['image_id_str', 'landmark_id']) # Drop rows with missing image_id or landmark_id
except FileNotFoundError:
    print("train.csv not found. Please ensure it's in the correct directory.")
    import sys
    sys.exit("Required 'train.csv' file not found. Please check the path.")
except KeyError:
    print("Error: 'id' or 'landmark_id' column not found in train.csv.")
    import sys
    sys.exit("Required columns not found in 'train.csv'. Please check the file content.")
except Exception as e:
    print(f"An unexpected error occurred while processing train.csv: {e}")
    import sys
    sys.exit("Error processing 'train.csv'.")


# 2. Initialize LabelEncoder (same as before)
lencoder = LabelEncoder()
# Fit on all unique landmark_ids in the dataframe after dropping NaNs
lencoder.fit(df['landmark_id'].astype(str).unique()) # Convert to string to handle potential float issues
num_classes = len(lencoder.classes_)

# 3. Create a Function to Load and Preprocess Each Image (same as before)
def load_and_preprocess_image(row_data):
    fname_stem = row_data['image_id_str']
    label = row_data['landmark_id']
    fname = fname_stem + ".jpg"

    # Ensure fname_stem has at least 3 characters before slicing
    if len(fname_stem) < 3:
        # print(f"WARNING: Invalid fname_stem format: {fname_stem}. Skipping image.")
        return None, None # Return None for both image and label

    f1 = fname_stem[0]
    f2 = fname_stem[1]
    f3 = fname_stem[2]

    relative_path = os.path.join(f1, f2, f3, fname)
    full_image_path = os.path.join(base_path, relative_path)

    im = cv2.imread(full_image_path)

    if im is None:
        # print(f"WARNING: Could not load image from: {full_image_path}. Returning None.")
        return None, None # Return None for both image and label
    else:
        im = cv2.resize(im, (224, 224)).astype(np.float32) / 255.0

    # Handle potential errors during label encoding
    try:
        label_encoded = lencoder.transform([str(int(label))])[0] # Ensure label is integer string
    except ValueError:
        # print(f"WARNING: Could not encode label: {label}. Skipping image.")
        return None, None

    return im, label_encoded

# 4. Create a Data Generator for tf.data.Dataset (same as before)
def data_generator(dataframe, batch_size, shuffle_data=False):
    num_samples = len(dataframe)
    indices = np.arange(num_samples)

    if shuffle_data:
        np.random.shuffle(indices)

    X_batch = []
    y_batch = []
    for i in indices:
        img, lbl = load_and_preprocess_image(dataframe.iloc[i])
        if img is not None and lbl is not None: # Only yield valid images and labels
            X_batch.append(img)
            y_batch.append(lbl)
            if len(X_batch) == batch_size:
                yield np.array(X_batch), np.array(y_batch)
                X_batch = []
                y_batch = []

    # Yield any remaining samples in the last batch
    if X_batch:
        yield np.array(X_batch), np.array(y_batch)


# 5. Split Data and Create tf.data.Dataset (same as before)
train_df = df.sample(frac=0.8, random_state=42).reset_index(drop=True)
validate_df = df.drop(train_df.index).reset_index(drop=True)

batch_size = 32
epochs = 5

output_signature = (
    tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(None,), dtype=tf.int32)
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_df, batch_size, shuffle_data=True),
    output_signature=output_signature
).prefetch(tf.data.AUTOTUNE)

validate_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(validate_df, batch_size, shuffle_data=False),
    output_signature=output_signature
).prefetch(tf.data.AUTOTUNE)

# 6. Build and Compile Your Keras Model using a pre-trained base
# Load a pre-trained model (e.g., MobileNetV2) without the top classification layer
base_model = applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the pre-trained layers
base_model.trainable = False

# Add new layers on top for your classification task
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x) # Bottleneck layer
predictions = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Optional: Print model summary to see the new number of parameters
model.summary()

# 7. Train the Model using model.fit() (same as before)
steps_per_epoch = int(np.ceil(len(train_df) / batch_size))
validation_steps = int(np.ceil(len(validate_df) / batch_size))

print(f"Training on {len(train_df)} samples, {steps_per_epoch} steps per epoch.")
print(f"Validating on {len(validate_df)} samples, {validation_steps} validation steps.")

history = model.fit(
    train_dataset,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=validate_dataset,
    validation_steps=validation_steps
)

model.save("Model.h5")
print("Model saved successfully as Model.h5")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 201,994,413 (770.55 MB)

 Trainable params: 199,736,429 (761.93 MB)

...

14008/14008 ━━━━━━━━━━━━━━━━━━━━ 120s 9ms/step
Epoch 5/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


14008/14008 ━━━━━━━━━━━━━━━━━━━━ 121s 9ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


Model saved successfully as Model.h5


In [3]:

batch_size = 8

errors = 0
good_preds = []
bad_pred = []

for X_val, y_val in validate_dataset:
    result = model.predict(X_val)
    cla = np.argmax(result, axis=1)

    for idx in range(X_val.shape[0]):
        if cla[idx] != y_val[idx]:
            errors = errors + 1

            bad_pred.append([idx, cla[idx], result[idx][cla[idx]]])
        else:
            good_preds.append([idx, cla[idx], result[idx][cla[idx]]])

print(f"Total errors: {errors}")
print(f"Accuracy: {1 - (errors / len(validate_df))}")


Total errors: 0
Accuracy: 1.0
